# Imports

In [1]:
import numpy as np
import pandas as pd
import random as rand

from keras.utils import to_categorical
from keras.layers import Dense, Activation, Conv2D,Conv2DTranspose, Dropout, Reshape, MaxPooling2D, Flatten
from keras.models import Sequential, load_model

from sklearn.model_selection import train_test_split

2023-08-05 17:13:50.296541: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-05 17:13:51.751291: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-05 17:13:51.754057: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-05 17:14:02.639637: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Data importing and pre-processing

In [2]:
df = pd.read_csv('dataset.csv', header=None)


df = df.values.reshape(60, 64, 64, 1)

labels = np.zeros(60)

x_real_train, x_real_test = train_test_split(df, test_size=0.2) #12 test values
y_real_train, y_real_test = train_test_split(labels, test_size=0.2)

# Generator Model

Create points in latent space to be fed into generator

In [3]:
def generate_generator_input(no_of_samples):
    input = np.random.randn(60*no_of_samples)

    input = input.reshape(60,no_of_samples)

    return input

In [4]:
def make_generator():
    model = Sequential()
    
    model.add(Dense(60*8*8, input_dim=60))
    model.add(Activation("relu"))
    model.add(Reshape((8,8,60)))
    
    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(8,8,60)))
    model.add(Activation("relu"))
  
    
    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(16,16,60)))
    model.add(Activation("relu"))

    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(32,32,60)))
    model.add(Activation("relu"))

    model.add(Dense(1))
    model.add(Activation("sigmoid"))

              
    model.summary()
    
    return model
    
    

# Discriminator Model

In [5]:

def make_discriminator():
    
    # 1st set of layers
    model = Sequential()
    model.add(Conv2D(128, kernel_size=5, padding="same", input_shape=(64,64,1)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    
    
    # output layer
    model.add(Dense(1)) # Binary classification (2 outputs), so only 1 dense layer needed
    model.add(Activation("sigmoid"))

    model.add(Flatten())
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model
    

# Training

First, create the models from the functions

In [6]:
gen_model = make_generator()
disc_model = make_discriminator()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3840)              234240    
                                                                 
 activation (Activation)     (None, 3840)              0         
                                                                 
 reshape (Reshape)           (None, 8, 8, 60)          0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 16, 16, 1)         61        
 anspose)                                                        
                                                                 
 activation_1 (Activation)   (None, 16, 16, 1)         0         
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 32, 32, 1)         2         
 Transpose)                                             

Next, create fake training data, and put it together with the real data.

In [7]:
gen_input = generate_generator_input(60)

print(gen_input.shape)

x_fake = gen_model.predict(gen_input)
y_fake = np.ones((60,), dtype=int)

x_fake = x_fake.reshape(60, 64, 64, 1)

    
x_fake_train, x_fake_test = train_test_split(x_fake, test_size=0.2)
y_fake_train, y_fake_test = train_test_split(y_fake, test_size=0.2)

print("shapes:")
print("x_fake_train : " + str(x_fake_train.shape))
print("x_fake_test : " + str(x_fake_test.shape))
print("y_fake_train : " + str(y_fake_train.shape))
print("y_fake_test : " + str(y_fake_test.shape))

print("x_real_train : " + str(x_real_train.shape))
print("x_real_test : " + str(x_real_test.shape))
print("y_real_train : " + str(y_real_train.shape))
print("y_real_test : " + str(y_real_test.shape))

x_train = np.concatenate((x_fake_train, x_real_train))
y_train = np.concatenate((y_fake_train, y_real_train))
x_test = np.concatenate((x_fake_test, x_real_test))
y_test = np.concatenate((y_fake_test, y_real_test))

(60, 60)
2/2 [==============================] - 0s 7ms/step
shapes:
x_fake_train : (48, 64, 64, 1)
x_fake_test : (12, 64, 64, 1)
y_fake_train : (48,)
y_fake_test : (12,)
x_real_train : (48, 64, 64, 1)
x_real_test : (12, 64, 64, 1)
y_real_train : (48,)
y_real_test : (12,)


2023-08-05 17:25:37.193210: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15698944 exceeds 10% of free system memory.
2023-08-05 17:25:37.194565: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15724544 exceeds 10% of free system memory.
2023-08-05 17:25:37.202163: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15698944 exceeds 10% of free system memory.
2023-08-05 17:25:37.202721: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15724544 exceeds 10% of free system memory.


Now, train the discriminator

Disc training parameters:

In [8]:
N_EPOCHS = 12
BATCH_SIZE = 4
VERBOSE = 1


In [9]:
print(x_train.shape)
print(y_train.shape)

disc_model.compile(loss="binary_crossentropy")
disc_model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS, verbose=VERBOSE, validation_data=(x_test, y_test))

(96, 64, 64, 1)
(96,)
Epoch 1/12
 6/24 [======>.......................] - ETA: 0s - loss: 1.5878

2023-08-05 17:25:43.174077: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 8388608 exceeds 10% of free system memory.


24/24 [==============================] - 1s 32ms/step - loss: 0.6496 - val_loss: 0.3398
Epoch 2/12
24/24 [==============================] - 1s 23ms/step - loss: 0.3282 - val_loss: 0.3055
Epoch 3/12
24/24 [==============================] - 1s 24ms/step - loss: 0.2855 - val_loss: 0.2708
Epoch 4/12
24/24 [==============================] - 1s 24ms/step - loss: 0.2491 - val_loss: 0.2214
Epoch 5/12
24/24 [==============================] - 1s 25ms/step - loss: 0.2200 - val_loss: 0.1844
Epoch 6/12
24/24 [==============================] - 1s 32ms/step - loss: 0.1677 - val_loss: 0.1555
Epoch 7/12
24/24 [==============================] - 1s 25ms/step - loss: 0.1404 - val_loss: 0.1256
Epoch 8/12
24/24 [==============================] - 1s 25ms/step - loss: 0.1118 - val_loss: 0.1004
Epoch 9/12
24/24 [==============================] - 1s 25ms/step - loss: 0.0876 - val_loss: 0.0827
Epoch 10/12
24/24 [==============================] - 1s 23ms/step - loss: 0.0681 - val_loss: 0.0651
Epoch 11/12
24/24 [=

Put models together into full GAN

In [10]:

disc_model.trainable = False

gan_model = Sequential()

gan_model.add(gen_model)
gan_model.add(disc_model)

gan_model.compile(loss='binary_crossentropy',optimizer='adam')

gan_model.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 64, 64, 1)         234307    
                                                                 
 sequential_1 (Sequential)   (None, 1024)              3457      
                                                                 
Total params: 237764 (928.77 KB)
Trainable params: 234307 (915.26 KB)
Non-trainable params: 3457 (13.50 KB)
_________________________________________________________________


Train GAN

In [11]:

# create some fake data as input

x_gan = generate_generator_input(60)
y_gan = np.ones((60))

gan_model.fit(x_gan, y_gan)



2/2 [==============================] - 2s 225ms/step - loss: 0.0454


Create map using generator

In [14]:
# create the input from noise

gen_input = generate_generator_input(1)

# create output
output_map = gen_model.predict(gen_input)

InvalidArgumentError: Graph execution error:

Detected at node 'sequential/activation/Relu' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/ubuntu/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/ubuntu/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/ubuntu/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 728, in start
      self.io_loop.start()
    File "/home/ubuntu/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
      self._run_once()


Output result graphically

In [15]:
print(output_map)

NameError: name 'output_map' is not defined